# Modelo Estadístico N-Gram

Es un modelo probabilístico que se entrena a través de un corpus. Este modelo es útil en muchas aplicaciones de procesamiento de lenguaje natural como reconocimiento de voz, traducción automática, predicción de texto, etc.

Básicamente, en un modelo "n-gram" se construye en base a la frecuencia que ocurre una secuencia de palabras en un texto para luego predecir la siguientes palabras.

Lo que haremos es crear un modelo "n-gram" sobre un corpus y luego, en base a dos palabras que le daremos al modelo, este intentará de predecir las siguientes palabras.

In [180]:
# Importamos la librería nltk
import nltk

El dataset que trabajaremos en esta ocasión será el corpus de "Reuters". Este contiene 10,788 noticias compuesto por un total de 1.3 millones de palabras. Estas noticias han sido clasificadas dentro de 90 categorías y agrupadas en dos conjuntos de datos: entrenamiento y pruebas.

In [181]:
# Descargamos el corpus "Reuters"
nltk.download("reuters")

[nltk_data] Downloading package reuters to /home/win7/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [182]:
# Y también descargaremos el paquete "punkt" que nos ayudará a tokenizar los textos
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/win7/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [183]:
# Importamos la librería de "reuters"
from nltk.corpus import reuters
# También la de "trigrams"
from nltk import trigrams
from nltk.util import ngrams
# Y finalmente algunas funciones para manejo de diccionarios y contadores
from collections import Counter, defaultdict

In [184]:
# Creamos una variable, en este caso diccionario de diccionarios que nos ayudará a guardar el conteo de las ocurrencias de cadenas de palabras.
model = defaultdict(lambda: defaultdict(lambda: 0))

In [185]:
# Ejemplo de una oración del corpus de "reuters"
reuters.sents()[100]
# model["Now", "it"]["'"]

['Now',
 'it',
 "'",
 's',
 'largely',
 'out',
 'of',
 'their',
 'hands',
 ',"',
 'said',
 'Kleinwort',
 'Benson',
 'Ltd',
 'financial',
 'analyst',
 'Simon',
 'Smithson',
 '.']

In [186]:
# Crearemos una matriz de coocurrencia a través del diccionario.

# Para cada oración del corpus "reuters" ...
for sentence in reuters.sents():
    # Para cada trigrama de una oración ...
    # pad_xxx significa que se agregará un token de inicio o de fin a la oración
    for w1, w2, w3, w4 in ngrams(sentence, 4, pad_right=True, pad_left=True):
        # Calculamos la frecuencia con la que ocurre cada combinación de trigramas en el conjunto de datos
        # Hay que verlo como si en la matriz, las filas son la secuencia w1, w2 y las columnas w3 tienen la palabra a "predecir"
        model[(w1, w2, w3)][w4] += 1 # pseudo probability

In [187]:
# Volvamos a ver los items de nuestro objeto
list(model.items())[105:110]

[(('term', 'Tokyo', "'"),
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>, {'s': 1})),
 (('Tokyo', "'", 's'),
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
              {'loss': 1,
               'latest': 1,
               'package': 1,
               'export': 1,
               'partners': 1,
               'foreign': 1,
               'failure': 4,
               'alleged': 4,
               'stock': 1,
               'prime': 1,
               'Shin': 1,
               'request': 1,
               'economic': 1})),
 (("'", 's', 'loss'),
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
              {'might': 1, '-': 1, 'to': 1, 'of': 4})),
 (('s', 'loss', 'might'),
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>, {'be': 1})),
 (('loss', 'might', 'be'),
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>, {'their': 1}))]

In [188]:
# Ahora recorremos cada secuencia w1, w2 o fila del modelo
for w1_w2 in model:
    # Y para cada secuencia w1, w2 contamos la cantidad de veces que esa secuencia se encuentra presente en el modelo
    total_count = float(sum(model[w1_w2].values()))
    # Y ese valor lo usamos para calcular las probabilidades de una palabra w3, dada las dos anteriores palabras
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [189]:
# Volvamos a ver los items de nuestro objeto
list(model.items())[105:110]

[(('term', 'Tokyo', "'"),
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>, {'s': 1.0})),
 (('Tokyo', "'", 's'),
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
              {'loss': 0.05263157894736842,
               'latest': 0.05263157894736842,
               'package': 0.05263157894736842,
               'export': 0.05263157894736842,
               'partners': 0.05263157894736842,
               'foreign': 0.05263157894736842,
               'failure': 0.21052631578947367,
               'alleged': 0.21052631578947367,
               'stock': 0.05263157894736842,
               'prime': 0.05263157894736842,
               'Shin': 0.05263157894736842,
               'request': 0.05263157894736842,
               'economic': 0.05263157894736842})),
 (("'", 's', 'loss'),
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
              {'might': 0.14285714285714285,
               '-': 0.14285714285714285,
               'to': 0.1428571428

In [190]:
# Otra oración ejemplo del corpus "reuters"
reuters.sents()[200]

['"',
 'The',
 'government',
 ',',
 'however',
 ',',
 'does',
 'not',
 'want',
 'to',
 'accelerate',
 'reducing',
 'the',
 'debt',
 'by',
 'making',
 'an',
 'excessive',
 'trade',
 'surplus',
 ',"',
 'he',
 'said',
 '.']

In [191]:
# Lo imprimimos de otra forma
print(' '.join(reuters.sents()[200]))

" The government , however , does not want to accelerate reducing the debt by making an excessive trade surplus ," he said .


In [192]:
# Probamos con una secuencia de dos palabras que se extrajo de la oración anterior
sorted(dict(model[("does", "not", "want")]).items(), key=lambda x:x[1], reverse=True)

[('to', 0.6470588235294118),
 ('a', 0.11764705882352941),
 ('for', 0.058823529411764705),
 ('area', 0.058823529411764705),
 ('sterling', 0.058823529411764705),
 ('them', 0.058823529411764705)]

In [193]:
# Otro ejemplo para la secuencia "the price", podemos ver todas las posibles palabras que pueden venir a continuación de esta secuencia y con sus
# respectivas probabilides de aparición
sorted(dict(model[("the", "price", "is")]).items(), key=lambda x:x[1], reverse=True)

[('fair', 0.25), ('very', 0.25), ('relatively', 0.25), ('based', 0.25)]

In [194]:
# Otro ejemplo con una secuencia que intencionalmente está mal escrita en inglés "the today"
sorted(dict(model[("the", "today", "was")]).items(), key=lambda x:x[1], reverse=True)

[]

In [195]:
# Importaremos la librería "random"
import random

In [196]:
# Elegiremos una secuencia de 2 palabras con la cuál se comenzará a crear una nueva oración
text = ["the", "price", "is"]
# Y también declararemos una variable que nos ayudará a determinar cuándo se acabó la oración
sentence_finished = False

In [197]:
model[tuple(text[-3:])].keys()

dict_keys(['fair', 'very', 'relatively', 'based'])

In [198]:
# Iteramos mientras la oración no haya terminado
while not sentence_finished:
    # generamos un número aleatorio del 0 al 1 que será nuestro threshold
    r = random.random()
    accumulator = .0
    # iteramos sobre el conjunto de posibles palabras que pueden venir luego de una secuencia de otras dos palabras (las ultimas 2 del texto)
    for word in model[tuple(text[-3:])].keys():
        # obtenemos su probabilidad y la sumamos al "accumulator"
        accumulator += model[tuple(text[-3:])][word]
        # si el "accumulator" es mayor a nuestro "threshold", añadimos la palabra al final del texto
        if accumulator >= r:
            text.append(word)
            break

    # Si las últimas dos palabras del texto es una secuencia de "None", se terminará el bucle
    if text[-3:] == [None, None, None]:
        sentence_finished = True

In [199]:
" ".join(text[:-3])

'the price is based on a reference price of 700 Ecus per tonne .'